In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
!pip install -q transformers accelerate



In [ ]:
import torch
from transformers import LlavaForConditionalGeneration, LlavaProcessor
from transformers import BitsAndBytesConfig
from PIL import Image


In [ ]:
import torch
from transformers import LlavaForConditionalGeneration, LlavaProcessor

model_id = "llava-hf/llava-1.5-7b-hf"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

processor = LlavaProcessor.from_pretrained(model_id)

print("Model loaded successfully in FP16 ✅")


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
image_path = list(uploaded.keys())[0]
image = Image.open(image_path).convert("RGB")

image


In [ ]:
prompt = """
<image>
You are a dermatology assistant.

Patient metadata:
Age: 65
Sex: male
Anatomical site: head/neck
Personal history of melanoma: True
Family history of melanoma: False
Fitzpatrick skin type: II

Is this lesion benign or malignant?
Answer with one word only.
"""


In [ ]:
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt"
).to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=50
)

response = processor.decode(output[0], skip_special_tokens=True)

print(response)


# **testing with 10 sample images**

In [ ]:
from google.colab import files
uploaded = files.upload()
# metadata.csv

In [ ]:
from google.colab import files
uploaded = files.upload()
# skin_img_sample.zip

In [ ]:
import zipfile

with zipfile.ZipFile("skin_images_sample.zip", 'r') as zip_ref:
    zip_ref.extractall("images")

print("Images extracted!")


In [ ]:
import os

print(os.listdir("images/img_sample(10-20)")[:10])



In [ ]:
import pandas as pd
import os

data = pd.read_csv("metadata.csv")

print("Metadata loaded:", data.shape)


In [ ]:
image_folder = "images/img_sample(10-20)"
image_files = set(os.listdir(image_folder))

data["image_filename"] = data["isic_id"] + ".jpg"

eval_data = data[data["image_filename"].isin(image_files)].copy()

print("Available evaluation samples:", len(eval_data))


In [ ]:
eval_data = eval_data.sample(10, random_state=42).reset_index(drop=True)

eval_data[["isic_id", "benign_malignant"]]


In [ ]:
from PIL import Image
import torch

image_folder = "images/img_sample(10-20)"

def safe_value(val):
    if pd.isna(val):
        return "Unknown"
    return val

def predict_llava(row):
    image_path = f"{image_folder}/{row['isic_id']}.jpg"
    image = Image.open(image_path).convert("RGB")

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": f"""
You are a dermatology expert.

Classify the lesion as benign or malignant.

Metadata:
Age: {row['age_approx']}
Sex: {row['sex']}
Location: {row['anatom_site_general']}
Personal history: {safe_value(row['personal_hx_mm'])}
Family history: {safe_value(row['family_hx_mm'])}
Skin type: {safe_value(row['fitzpatrick_skin_type'])}

Answer with ONLY one word: benign or malignant.
"""}
            ]
        }
    ]

    #  Create formatted chat text
    formatted_prompt = processor.apply_chat_template(
        conversation,
        add_generation_prompt=True
    )

    #  Now tokenize properly with image
    inputs = processor(
        text=formatted_prompt,
        images=image,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=False
    )

    response = processor.decode(output[0], skip_special_tokens=True)

    torch.cuda.empty_cache()

    return response



In [ ]:
predictions = []

for i in range(len(eval_data)):
    print(f"\nProcessing {i+1}/{len(eval_data)}")
    pred = predict_llava(eval_data.loc[i])
    print("Raw prediction:", pred)
    predictions.append(pred)

eval_data["llava_prediction"] = predictions


In [ ]:
def clean_prediction(text):
    text = text.lower()

    # Look only at assistant response part
    if "assistant:" in text:
        text = text.split("assistant:")[-1]

    if "malignant" in text:
        return "malignant"
    elif "benign" in text:
        return "benign"
    else:
        return "unknown"

eval_data["llava_clean"] = eval_data["llava_prediction"].apply(clean_prediction)


In [ ]:
# “Zero-shot evaluation of LLaVA-7B on dermoscopic lesion classification showed majority-class bias and lack of domain generalization,
#  achieving near-baseline accuracy compared to structured ML models.”

In [ ]:
accuracy = (eval_data["llava_clean"] == eval_data["benign_malignant"]).mean()

print("LLaVA Accuracy:", accuracy)


In [ ]:
print(eval_data.columns)


In [ ]:
eval_data[["isic_id", "benign_malignant", "llava_clean"]]
